In [47]:
import pandas as pd
import openpyxl
import numpy as np
#from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import scipy.spatial.distance as dis
import scipy.cluster.hierarchy as hierarchy
import scipy.spatial.distance as dist
import copy
import os
import sys
import folium
from folium.plugins import FastMarkerCluster

In [48]:
currentpath=os.getcwd()
basepath=currentpath.split("/")
basepath="/".join(basepath[0:basepath.index("test")+1])+"/data/ryukyu4/{}"

In [49]:
dORe=int(input("発音構造か発音変化か：0,1"))
gnumber=int(input("グラム数の指定"))
porder=int(input("パターンの入力：all=0,cvc=1,vcv=2"))

発音構造か発音変化か：0,10
グラム数の指定1
パターンの入力：all=0,cvc=1,vcv=22


In [50]:
if dORe==0:
    dORe="E"
    pattern="counter"
elif dORe==1:
    dORe="D"
    pattern="pattern"

In [51]:
if porder==0:
    porder="all"
    porder2="A"
elif porder==1:
    porder="cvc"
    porder2="C"
elif porder==2:
    porder="vcv"
    porder2="V"

In [52]:
rflocate = basepath.format("parameter/locations.xlsx")
rfword   = basepath.format("parameter/sheetlist.xlsx")
picpath=basepath.format("/pic/{}")

In [53]:
locateDf = pd.read_excel(rflocate, sheet_name=0, header=0, index_col=0)
wordDf  = pd.read_excel(rfword , sheet_name=0, header=0, index_col=0)

In [54]:
symbols=list(locateDf.iloc[:,0])
locates = list(locateDf.iloc[:,2])
hyouzi = list(locateDf.iloc[:,1])
words   = list(wordDf.iloc[:,0])

In [55]:
rflabel=basepath.format("gram{}/{}/{}Label.xlsx").format(gnumber,pattern,porder)
rfid=basepath.format("parameter/テスト用数値一覧・地点_緯度経度20190313.csv")
labeldf=pd.read_excel(rflabel,index_col=0)
idodf=pd.read_csv(rfid,header=0,index_col=0)

In [56]:
labels=list(labeldf.iloc[:,0].values)
idokeido=idodf.loc["旧笠利町笠利":,["緯度","経度"]]

In [57]:
color_set=list(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [58]:
truelabel=[0 for i in locates]
locindex=[0,16,32,73,81,93,95]
for i in range(6):
    for j in range(locindex[i],locindex[i+1]):
        truelabel[j]=i

# ラベル一致の確認

In [59]:
print(labels)
print(truelabel)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 1, 5, 5, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5]


In [60]:
truelabels2=[0 for i in locates]
asignlabels=[0 for i in range(6)]
locindex=[0,16,32,73,81,93,95]

# asignlabels=[0,1,2,3,4,5] #D3A
# asignlabels=[0,1,2,3,4,5] #D3A2
# asignlabels=[0,2,1,4,3,5] #D3C
# asignlabels=[0,1,2,3,4,5] #D3V
# asignlabels=[0,4,2,1,3,5] #D2A
# asignlabels=[0,2,1,3,4,5] #D2A2
# asignlabels=[0,5,1,4,2,3] #D2C
# asignlabels=[0,1,2,3,5,4] #D2V
# asignlabels=[0,4,1,2,3,5] #D1A2
# asignlabels=[0,1,3,2,5,4] #D1C
asignlabels=[0,5,4,1,2,3] #D1V
# asignlabels=[0,2,1,4,3,5] #E3C
# asignlabels=[0,1,2,3,4,5] #E3V
# asignlabels=[0,5,1,4,2,3] #E2C
# asignlabels=[0,4,2,1,5,3] #E2V
# asignlabels=[0,4,1,2,5,3] #E1C
# asignlabels=[0,4,5,1,2,3] #E1V
for i in range(6):
    for j,label in enumerate(labels):
        if i==label:
            truelabels2[j]=asignlabels[i]

print(labels)
print(truelabels2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 1, 5, 5, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 4, 4, 4, 5, 5, 5, 1, 1, 5, 5, 5, 5, 1, 5, 5, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 3, 3, 5, 3, 3, 5, 5]


In [61]:
_map = folium.Map(location=list(idokeido.iloc[25,:].values),
                  attr="Stamen Watercolor")
for i,(idkd,label) in enumerate(zip(list(idokeido.values),truelabels2)):
    folium.CircleMarker(
        popup='{}'.format(hyouzi[i]),
        location= idkd,
        radius=4,
        color=color_set[label],
        fill=True,
        fill_color=color_set[label]
    ).add_to(_map)

# _map.save(basepath.format("gram{}/pic/map-N{}{}-{}.html").format(gnumber,gnumber,porder,dore))

# _map.save(basepath.format("gram3/pic/map-N3-D.html"))
# _map.save(picpath.format("map-N{}-D.html".format(gnumber)))v

In [62]:
_map

In [38]:
savename=basepath.format("gram{}/pic/map-N{}{}-{}.html").format(gnumber,gnumber,porder,dORe)
# savename=basepath.format("gram{}/pic/map2-N{}{}-{}.html").format(gnumber,gnumber,porder2,dORe)
_map.save(savename)

In [18]:
# new_color=[0 for i in range(6)]
# new_color_ind=[4,3,0,5,2,1]
# for i in range(6):
#     new_color[i]=color_set[new_color_ind[i]]

In [19]:
# plt.figure(figsize=(60,25))
# figtitle='Dendrogram for misalignment of sound between regions'
# plt.rcParams['font.family'] = 'IPAexGothic'
# plt.rcParams['font.size'] = 50
# plt.rcParams['lines.linewidth'] = 4
# plt.title(figtitle, fontsize=60)
# plt.xlabel('観測地点', fontsize=55)
# plt.ylabel('クラスタ間距離(Cos類似度)', fontsize=55)
# hierarchy.set_link_color_palette(new_color)
# # hierarchy.set_link_color_palette(['purple', 'lawngreen', 'green', 'blue', 'orange', 'red']) # ６クラスタまでの色を指定
# hierarchy.dendrogram(z, leaf_font_size=35., # 横軸の文字の大きさを指定
#                         color_threshold=0.005, # ユークリッド平方距離が７以上を同色で表示
#                         above_threshold_color='black',
#                         labels=tuple(locates)) # ユークリッド平方距離が７以上を黒色に指定
# plt.tight_layout()
# # plt.savefig(picpath.format("dend-N{}-D.png".format(gnumber)))
# plt.show()
# plt.close()

In [39]:
labelloc=[0 for i in range(6)]
for i in range(6):
    labelloc[i]=[j for j,label in enumerate(truelabels2) if i == label]

In [40]:
print(labelloc)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [19, 20, 25, 28, 29, 30, 31], [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72], [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 88, 89, 91, 92], [13, 14, 15], [10, 11, 12, 16, 17, 18, 21, 22, 23, 24, 26, 27, 44, 55, 86, 87, 90, 93, 94]]


In [41]:
wf=basepath.format("gram{}/{}/lpt{}.xlsx").format(gnumber,pattern,porder2)
rfpattern= basepath.format("gram{}/{}/{}.xlsx").format(gnumber,pattern,porder)

In [42]:
ptDf=pd.read_excel(rfpattern, sheet_name=0, header=0, index_col=0)

In [43]:
nonind=ptDf.index.str.contains('-9')
ptDf=ptDf.loc[~nonind,:]
indexName=ptDf.index
ptDf=ptDf.T

In [44]:
a=list(range(6))
for i in range(6):
    tmp=ptDf.iloc[labelloc[i],:].values
    a[i]=tmp.mean(axis=0)
a=np.array(a)
a=a.T

In [45]:
wdf=pd.DataFrame(a,index=indexName,columns=["奄美","北部","南部","宮古","八重山","与那国"])

In [46]:
with pd.ExcelWriter(wf, engine='openpyxl') as writer:
    wdf.to_excel(writer,sheet_name="ラベル")

In [63]:
wdf2=pd.DataFrame(truelabels2,index=truelabel,columns=["ラベル"])
wflabels=basepath.format("gram{}/{}/label{}.xlsx").format(gnumber,pattern,porder2)
with pd.ExcelWriter(wflabels, engine='openpyxl') as writer:
    wdf2.to_excel(writer,sheet_name="ラベル")